In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics

plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# Mnist dataset fpr Lenet
batch_size = 128
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
training_id = 10
model_short_name = 'lenet5'
framework = 'pytorch'

In [5]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/pytorch/lenet5/'

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_lenet5-mnist_2021-11-01_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

252295

In [6]:
size1 = os.path.getsize(path+model_name+'.pth')
size1

252295

In [7]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [8]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/errors/'

In [9]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [10]:
def to_onnx(i, x, data_writer_error, data_writer_error2, data_writer_run, batch_size):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    traced_model = torch.jit.trace(model, x)
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_ = time.time()
    since_1 = time.time()
    torch_out = model(x)
    t_elapsed_1 = time.time() - since_1
    
    # Export the model
    if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
        os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
    if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
        os.makedirs(coreml_path+framework+"/{}".format(model_short_name))
        
    since_1 = time.time()
    
    since_onnx = time.time()
    torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    onnx.checker.check_model(onnx_model)
    size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    t_elapsed_3 = time.time() - since_1
    t_elapsed_ = time.time() - since_
    t_elapsed_onnx = time.time() - since_onnx
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    abs_err = np.absolute(to_numpy(torch_out)-ort_outs[0])
    rel_err = np.absolute(to_numpy(torch_out)-ort_outs[0])/ np.absolute(ort_outs[0])
    #print('Batch: ', i, abs_err, rel_err)
    

    
    ## Converting the coremltool
    since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    t_elapsed_4 = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    t_elapsed_5 = time.time() - since_1
    
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    #print(name_1)

    size3 = os.path.getsize(coreml_path+framework+'/{}/{}.mlmodel'.format(model_short_name, model_name))
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    t_elapsed_6 = time.time() - since_1
    t_elapsed2_ = time.time() - since_
    
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    abs_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)
    rel_err2 = np.absolute(to_numpy(torch_out)-coreml_array_output)/ np.absolute(coreml_array_output)
    
    for j in range (len(abs_err)):
        for k in range(len(abs_err[j])): 
            data_writer_error.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err[j][k], rel_err[j][k]])
    
    
    for j in range (len(abs_err2)):
        for k in range(len(abs_err2[j])): 
            data_writer_error2.writerow([model_short_name,framework, training_id, model_name, batch_size, i, abs_err2[j][k], rel_err2[j][k]])
      
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size2, t_elapsed_1, t_elapsed_3,  t_elapsed_2,'', t_elapsed_, np.mean(abs_err), np.median(abs_err), np.min(abs_err), np.max(abs_err), np.mean(rel_err), np.median(rel_err), np.min(rel_err), np.max(rel_err)])
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size3, t_elapsed_1, t_elapsed_6, t_elapsed_4, t_elapsed_5, (t_elapsed2_-t_elapsed_onnx), np.mean(abs_err2), np.median(abs_err2), np.min(abs_err2), np.max(abs_err2), np.mean(rel_err2), np.median(rel_err2), np.min(rel_err2), np.max(rel_err2)])

In [11]:
def _lets_convert(data, data_writer_error, data_writer_error2, data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    for i, (images, labels) in enumerate(data):
        to_onnx(i, images,data_writer_error, data_writer_error2, data_writer_run, batch_size)
        if i == 20: 
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [12]:
#model_name = 'letnet5-keras'
import pandas as pd 
def run_experiment(model_short_name, model_name):
    if not os.path.exists(error_path+framework+"/{}".format(model_short_name)): 
        os.makedirs(error_path+framework+"/{}".format(model_short_name))

    data_file_error = open(error_path + framework+'/{}/onnx_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error = csv.writer(data_file_error, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_error2 = open(error_path + framework+'/{}/mlmore_error_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_error2 = csv.writer(data_file_error2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_error2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','absolute_error', 'relative_error'])


    data_file_run = open(error_path + framework+'/{}/runtime_metrics_{}.csv'.format(model_short_name, model_name), mode='w', newline='',
                                      encoding='utf-8')
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','converter','original_size', 'converted_size', 'original_infererence_time', 'converted_infererence_time', 'conversion_time', 'saving_converted_time', 'overral_time', 'avg_abs_error', 'median_abs_error', 'min_abs_error', 'max_abs_error','avg_rel_error', 'median_rel_error', 'min_rel_error', 'max_rel_error'])
    
    #for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150,200, 250,300,350, 400, 450, 500]:
    for batch_size in [1, 5,10,50,100,128, 150,200, 250,300,350, 400, 450, 500]:
        print("################ Batch size: ", batch_size)
        #trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
        valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)
        
        _lets_convert(valloader, data_writer_error, data_writer_error2, data_writer_run, batch_size)
    data_file_error.close()
    data_file_error2.close()
    data_file_run.close()

In [13]:
run_experiment(model_short_name, model_name)

################ Batch size:  1
converting for batch:  0

*********




Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'input.1', of the source model, has been renamed to 'input_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '57', of the source model, has been renamed to 'var_57' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1557.30 ops/s]


converting for batch:  1

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 2256.97 ops/s]


converting for batch:  2

*********




...

converting for batch:  18

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1619.05 ops/s]


converting for batch:  19

*********




Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 2976.79 ops/s]


Conversion complete in 0m 32s


In [10]:
for i, (images, labels) in enumerate(valloader):
    x_test, y_test = images, labels
    break

In [14]:
# Export the model
if not os.path.exists(onnx_path+framework+"/{}".format(model_short_name)):
    os.makedirs(onnx_path+framework+"/{}".format(model_short_name))
if not os.path.exists(coreml_path+framework+"/{}".format(model_short_name)):
    os.makedirs(coreml_path+framework+"/{}".format(model_short_name))

In [15]:
torch.onnx._export(model,               # model being run
                  x_test,                         # model input (or a tuple for multiple inputs)
                  onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

tensor([[ -6.6358,   1.2895,   0.4550,  ...,  21.1323,  -5.8902,   4.0097],
        [  0.6449,   6.2741,  34.7097,  ...,  -0.3277,  -1.8668,  -7.1835],
        [ -7.5961,  15.3428,  -2.6259,  ...,   0.8016,   2.4336,  -0.9590],
        ...,
        [ -2.7858,  -5.1429,   0.6817,  ...,  -1.0616,   0.7974,  13.7856],
        [ 22.0681,  -8.9676,   1.9604,  ...,  -7.2898,  -3.2032,   0.6012],
        [ -5.8501,  -7.5255, -12.7759,  ...,  -4.8417,   3.1106,   4.5962]],
       grad_fn=<AddmmBackward>)

In [12]:
from onnx2keras import onnx_to_keras
from onnx_tf.backend import prepare

In [17]:
onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
tf_rep = prepare(onnx_model)  # prepare tf representation
print(tf_rep.inputs)
k_model = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight 1 with shape (6, 1, 5, 5).
DEBUG:onnx2keras:Found weight 2 with shape (6,).
DEBUG:onnx2keras:Found weight 3 with shape (16, 6, 5, 5).
DEBUG:onnx2keras:Found weight 4 with shape (16,).
DEBUG:onnx2keras:Found weight 5 with shape (120, 400).
DEBUG:onnx2keras:Found weight 6 with shape (120,).
DEBUG:onnx2keras:Found weight 7 with shape (84, 120).
DEBUG:onnx2keras:Found weight 8 with shape (84,).
DEBUG:onnx2keras:Found weight 9 with shape (10, 84).
DEBUG:onnx2keras:Found weight 10 with shape (10,).
DEBUG:onnx2keras:Found input input with shape [1, 28, 28]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:nod

['input']


DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 6, 28, 28), dtype=tf.float32, name=None), name='11/BiasAdd:0', description="created by layer '11'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 12
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 11).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 6, 28, 28), dtype=tf.float32, name=None), name='12/Relu:0', description="created by layer '12'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Pad
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'mode': b'constant', 'pads': [0, 0, 0, 0, 0, 0, 0, 0], 'value': 0.0, 'change_orde

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]


DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 16, 5, 5), dtype=tf.float32, name=None), name='18/AvgPool:0', description="created by layer '18'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Flatten
DEBUG:onnx2keras:node_name: 19
DEBUG:onnx2keras:node_params: {'axis': 1, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 18).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:flatten:Convert inputs to Keras/TF layers if needed.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='19/Reshape:0', description="created by layer '19'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Gemm
DEBUG:onnx2keras:node

In [18]:
from tensorflow import keras
k_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
import tensorflow.keras as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
import tf2onnx
import numpy as np
import math

In [20]:
batch_size = 500 # Number of images processed at once
nb_classes = 10  # 10 Digits from 0 to 9

# Dimensionen of the input images (28x28 pixel)
img_rows, img_cols = 28, 28

# Load image data with labels, split into test and training set 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# reshape images in 4D tensor (N images, 28 rows, 28 columns, 1 channel) 
# rescale pixels range from [0, 255] to [0, 1]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape: ', X_train.shape)
print(X_train.shape[0], "training samples")
print(X_test.shape[0], "test samples")

# convert digit labels (0-9) in one-hot encoded binary vectors. 
# These correspond to the training/test labels at the output of the net. 
Y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
Y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
print("One-hot encoding: {}".format(Y_train[0, :]))

X_train shape:  (60000, 28, 28, 1)
60000 training samples
10000 test samples
One-hot encoding: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [21]:
history = k_model.fit(X_train, Y_train, batch_size=32, epochs=2, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/2


ValueError: in user code:

    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:838 run_step  **
 

In [11]:
traced_model = torch.jit.trace(model, x_test)

In [14]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
coreml_model = coremltools.convert(
    traced_model,
    inputs=[coremltools.TensorType(shape=x_test.shape)]
 )

Running MIL Common passes:   0%|          | 0/33 [00:00<?, ? passes/s]/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:101: UserWarning: Input, 'input.1', of the source model, has been renamed to 'input_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/opt/anaconda3/lib/python3.7/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '57', of the source model, has been renamed to 'var_57' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 1833.23 ops/s]


In [15]:
split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
name_1 = split_[0].replace('name: "', '')
name_1 = name_1.replace('"', '')

In [19]:
output_dict_test = coreml_model.predict({name_1:to_numpy(x_test)})

['var_57']

In [24]:
output_dict_test[list(output_dict_test.keys())[0]]

array([[ -8.23903   ,  -2.739309  ,  -0.57399404, ...,  25.12432   ,
         -5.8766212 ,   7.7804904 ],
       [  1.2041439 ,   1.7744694 ,  23.83385   , ...,  -1.4935052 ,
          0.73381054, -20.842278  ],
       [ -3.5446525 ,  17.608837  ,  -5.0224304 , ...,  -1.1348255 ,
          2.4645977 ,  -2.9240925 ],
       ...,
       [ -3.667636  ,   3.369899  ,  -5.359874  , ...,  -1.9468975 ,
          4.964617  ,   2.219088  ],
       [ 17.240082  ,  -7.4075713 ,  -1.3843741 , ...,  -6.6437745 ,
          1.8500232 ,  -1.9735501 ],
       [  2.2919703 ,  -7.3513675 ,  -7.3908825 , ...,  -7.230786  ,
          0.21587932,  -5.5876713 ]], dtype=float32)